In [ ]:
from ipyleaflet import Map, Marker
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from datetime import datetime, timedelta
from pydap.cas.urs import setup_session
#from sidecar import Sidecar
import sys
sys.path.append('../py')
from hydromap import Flow, get_polygon
from delineate import delineate
from misc import adjust_bbox, aggregate_da, pixel_area
import gcsfs

In [ ]:
center = (-10, -60)
zoom = 4
m = Map(center=center, zoom=zoom, interpolation='nearest')
#sc = Sidecar(title='Map')
#with sc:
#    display(m)
m

In [ ]:
message = widgets.Label()
display(message)
flow = Flow(m, message)
widgets.interact(flow.show, width=widgets.FloatSlider(min=3/1200,max=0.1,step=1/1200,value=0.05))
m.on_interaction(flow.show)

# Right-click menu
A menu pops up when you right-click on a location. It allows you to:
- show/hide flow accumulation. Flow accumulation is only visible on a square around the mouse position, because it takes too much memory for the browser to show it all, and also because it is context dependent: the color map will fit the range of flow accumulation values that are displayed. The width of this square can be changed with the slider above (values are in degrees). Please note that the flow accumulation image might not correspond to the rivers you see on the map, especially far from the mouse position, first because they come from different sources, and second because the image is displayed as-is (EPSG:4326) and not reprojected to Web Mercator (EPSG:3857). The difference is small enough near the mouse position anyway, and the purpose of showing flow accumulation is just to check that you are on a river before delineating a watershsed.
- delineate a watershed. Delineation will start at the pixel the mouse is positioned on. You can zoom in until you make sure you are on the river of interest, and you can also check the lat/lon coordinates and flow accumulation numbers displayed above.

# Marker

Use the following cell to display a marker on the map at a particular location. You can then drag it with the mouse and get its new location back with the next cell.

In [ ]:
latlon = -19.043, -63.785
marker = Marker(location=latlon)
m.add_layer(marker)

In [ ]:
marker.location

In [ ]:
lat, lon = -19.04560829389358, -63.786876992334314 # R_amz_gde_env_0564_02
ws = delineate(lat, lon)

In [ ]:
polygon = get_polygon(ws, 0)
m.add_layer(polygon)

In [ ]:
pix_deg_flow = 1 / 1200
pix_deg_trmm = 0.25
ratio = int(pix_deg_trmm / pix_deg_flow)
da_mask_trmm = []
for j, mask in enumerate (ws['mask']):
    lat = np.array([ws['latlon'][j][0] - (i + 0.5) * pix_deg_flow for i in range(mask.shape[0])])
    lon = np.array([ws['latlon'][j][1] + (i + 0.5) * pix_deg_flow for i in range(mask.shape[1])])
    da1 = xr.DataArray(mask, coords=[lat, lon], dims=['lat', 'lon'])
    da2 = adjust_bbox(da1, {'lat': (pix_deg_trmm, -pix_deg_flow), 'lon': (pix_deg_trmm, pix_deg_flow)})
    da3 = aggregate_da(da2, {'lat': ratio, 'lon': ratio}) / (ratio * ratio)
    da3 = da3.rename({'lat_agg': 'lat', 'lon_agg': 'lon'})
    da3.lon.values = np.round(da3.lon.values, 2)
    da3.lat.values = np.round(da3.lat.values, 2)
    da_mask_trmm.append(da3)
da_mask_trmm = xr.concat(da_mask_trmm, 'ws').assign_coords(ws=ws['label'])

In [ ]:
da_mask_trmm.sum(['ws']).plot.imshow(figsize=(15, 10))

In [ ]:
da_mask = []
for j, mask in enumerate (ws['mask']):
    lat = np.array([ws['latlon'][j][0] - (i + 0.5) * pix_deg_flow for i in range(mask.shape[0])])
    lon = np.array([ws['latlon'][j][1] + (i + 0.5) * pix_deg_flow for i in range(mask.shape[1])])
    da1 = xr.DataArray(mask, coords=[lat, lon], dims=['lat', 'lon'])
    da2 = adjust_bbox(da1, {'lat': (pix_deg_trmm, -pix_deg_flow), 'lon': (pix_deg_trmm, pix_deg_flow)})
    da_mask.append(da2*np.random.rand())

def reindex(arrays, dims):
    tolerance = 0.1 * pix_deg_flow
    vmin, vmax = {}, {}
    for d in dims:
        vmin[d] = np.inf
        vmax[d] = -np.inf
    for da in arrays:
        this_vmin, this_vmax = {}, {}
        for d in dims:
            vmin[d] = min(vmin[d], np.min(da[d]).values)
            vmax[d] = max(vmax[d], np.max(da[d]).values)
    coord = {}
    for d in dims:
        if arrays[0][d].values[1] - arrays[0][d].values[0] > 0: # increasing
            coord[d] = np.arange(vmin[d], vmax[d]+tolerance, pix_deg_flow)
        else:
            coord[d] = np.arange(vmax[d], vmin[d]-tolerance, -pix_deg_flow)
    for i in range(len(arrays)):
        arrays[i] = arrays[i].reindex({d: coord[d] for d in dims}, method='nearest', tolerance=tolerance)

reindex(da_mask, ['lat', 'lon'])
da_mask = xr.concat(da_mask, 'ws').assign_coords(ws=ws['label'])

In [ ]:
da_mask.sum(['ws']).plot.imshow(figsize=(15, 10))

In [ ]:
ds_trmm = xr.open_zarr(gcsfs.GCSMap('pangeo-data/trmm_3b42rt'))

In [ ]:
da_area = pixel_area(pix_deg_trmm)

In [ ]:
da_mask_trmm = da_area.reindex_like(da_mask_trmm, method='nearest', tolerance=0.01) * da_mask_trmm
da_mask_trmm = da_mask_trmm / da_mask_trmm.sum(['lat', 'lon'])

In [ ]:
p = (ds_trmm['precipitation'].reindex_like(da_mask_trmm, method='nearest', tolerance=0.01) * da_mask_trmm).sum(['lat', 'lon'])

In [ ]:
with ProgressBar():
    p = p.compute()

In [ ]:
p.sum(['ws']).plot()